In [1]:
!pip install transformers
!pip install pdfplumber
!pip install torch
!pip install sentence-transformers
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 17.3 MB/s eta 0:00:00


In [2]:
import pdfplumber
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import torch
import faiss


In [3]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving BDM Capstone Project Document for 2024.pdf to BDM Capstone Project Document for 2024.pdf


In [4]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

document_text = extract_text_from_pdf(pdf_path)
print(document_text[:1000])


BDM Capstone Project Document for 2024 students
This document has been prepared jointly by Dr. Aaditya Chandel and Dr. Ashwin J. Baliga.
All have contributed equally towards the preparation of this document.
Do go through all the live sessions, FAQs, documents, and the reference books, to facilitate
better understanding.
Any report submitted without name and or roll number is desk rejected!
Every submission should strictly adhere to the rubrics. If it doesn’t adhere to it, you end up
losing marks. Format and structure your reports the way we have done here.
We would prefer Times New Roman 12, 1.5 spacing and left/ right justification of fonts. Also,
you need to ensure that the report is written well and do not mix active/ passive voice. Project is
not just showing data set and graphs. A good report should explain all the elements in a very neat
manner. Using commas and full stops wherever possible. Also, ensure you connect one section
with the next. There must be a connection when you 

In [5]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
sentences = document_text.split(". ")
sentence_embeddings = embedder.encode(sentences, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# def get_best_answer(question, sentences, sentence_embeddings, embedder):
#     question_embedding = embedder.encode(question, convert_to_tensor=True).cpu().numpy()
#     question_embedding = question_embedding.reshape(1, -1)
#     index = faiss.IndexFlatL2(sentence_embeddings.shape[1])
#     index.add(sentence_embeddings.cpu().numpy())
#     search_result = index.search(question_embedding, 3)
#     if len(search_result) == 2:
#         _, indices = search_result
#         context = " ".join([sentences[i] for i in indices[0]])
#     else:
#         print("Error: Could not retrieve results from FAISS search.")
#         return "I'm sorry, I couldn't find an answer to that question."
#     answer = qa_pipeline({'question': question, 'context': context})
#     return answer['answer']


# def get_best_answer(question, sentences, sentence_embeddings, embedder, top_k=5, context_size=100):
#     question_embedding = embedder.encode(question, convert_to_tensor=True).cpu().numpy()
#     question_embedding = question_embedding.reshape(1, -1)
#     index = faiss.IndexFlatL2(sentence_embeddings.shape[1])
#     index.add(sentence_embeddings.cpu().numpy())
#     _, indices = index.search(question_embedding, top_k)
#     context = []
#     for i in indices[0]:
#         start = max(0, i - context_size // 2)
#         end = min(len(sentences), i + context_size // 2)
#         context.append(" ".join(sentences[start:end]))
#     context = " ".join(context)
#     answer = qa_pipeline({'question': question, 'context': context})
#     return answer['answer']

def get_best_answer(question, sentences, sentence_embeddings, embedder):
    question_embedding = embedder.encode(question, convert_to_tensor=True).cpu().numpy()
    question_embedding = question_embedding.reshape(1, -1)
    index = faiss.IndexFlatL2(sentence_embeddings.shape[1])
    index.add(sentence_embeddings.cpu().numpy())
    search_result = index.search(question_embedding, 3)
    if len(search_result) == 2:
        _, indices = search_result
        context = " ".join([sentences[i] for i in indices[0]])
    else:
        print("Error: Could not retrieve results from FAISS search.")
        return "I'm sorry, I couldn't find an answer to that question."
    input_text = f"Question: {question} Context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
def format_answer(answer, words_per_line=15):
    words = answer.split()
    formatted_answer = "\n".join(" ".join(words[i:i+words_per_line]) for i in range(0, len(words), words_per_line))
    return formatted_answer

In [9]:
print("Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'stop':
        print("Chatbot: Goodbye!")
        break
    answer = get_best_answer(user_input, sentences, sentence_embeddings, embedder)
    formatted_answer = format_answer(answer, words_per_line=15)
    print(f"Chatbot: {formatted_answer}")

Chatbot is ready! Ask questions about the PDF content, or type 'stop' to exit.
You: who wrote this document
Chatbot: Dr Baliga
You: how to do data collection
Chatbot: It is the objectives that determine the methodology and not the other way round. Once
this is done provide information on how you plan to collect the data For instance,
data collection- 2months (mention dates), final submission (3 months) and give a breakup of work
you intend to do.
You: what should be my document's font size and other specifcations
Chatbot: Times New Roman 12, 1.5 spacing and left/ right justification of fonts
You: stop
Chatbot: Goodbye!
